# Godot 翻译
使用 Kotlin Notebook

## 物料准备
[Godot Pck Tool](https://github.com/hhyyrylainen/GodotPckTool)

提取资源文件至 extracted 目录
```
.\godotpcktool.exe .\vcb.pck -a e -o extracted
```
打包 extracted 目录的资源至 vcb.pck 文件
```
.\godotpcktool.exe .\vcb.pck -a a extracted --remove-prefix extracted
```

In [9]:
// 提取出的资源目录
val extractedDirPath = "C:\\Users\\Xeu\\3D Objects\\vcb\\extracted"

 // 工作目录：vcb.json 与 vcb_cn.json 保存的目录
 val workDirPath = "C:\\Users\\Xeu\\3D Objects\\vcb_cn\\mapping"

 // 原文 json 文件名
 val mappingKeyFileName = "vcb.json"

 // 译文 json 文件名
 val mappingValueFileName = "vcb_cn.json"
 // 旧版本 json 文件名
 val mappingKeyFileNameOld = "vcb_0.3.json"

 // 旧版本译文 json 文件名
 val mappingValueFileNameOld = "vcb_cn_0.3.json"

In [10]:
// 提取所有文本
val regex = """(text|hint_tooltip|title|description) = "([\W\w]*?[^\\])"""".toRegex()

In [11]:
// 初始化
import java.io.File
val folder = File(extractedDirPath)

In [5]:
// 已有 vcb.json 文件后不需要再次提取
// 提取所有文本内容至指定文件(mappingKeyFileName指定的文件)

import kotlinx.serialization.json.JsonArray
import kotlinx.serialization.json.JsonPrimitive

val mappingKeys = folder.walk()
    .filter { (it.isFile && it.extension == "tscn") }
    .map { tscn ->
        val data = tscn.readText()
        regex.findAll(data).map {
            it.groupValues[2]
        }.toList()
    }.flatMap {
        it
    }.toSet() // toSet 去除重复文本
val json = JsonArray(mappingKeys.map { JsonPrimitive(it) }).toString()
val output = File(workDirPath, mappingKeyFileName)
output.writeText(json)

In [6]:
// 翻译过程：将 vcb.json 文件拷贝一份到 vcb_cn.json，然后使用其他工具(如 ChatGPT )进行翻译

In [4]:
// 增量翻译：使用过去的映射表翻译当前的翻译文件
@file:DependsOn("com.google.code.gson:gson:2.10.1")

import com.google.gson.Gson
import kotlinx.serialization.json.jsonArray
import kotlinx.serialization.json.jsonObject

val untranslatedFile = File(workDirPath, mappingKeyFileName)
val untranslatedText = untranslatedFile.readText()
val untranslatedList = Gson().fromJson<List<String>>(untranslatedText, ArrayList::class.java)

val untranslatedOldFile = File(workDirPath, mappingKeyFileNameOld)
val untranslatedOldText = untranslatedOldFile.readText()
val untranslatedOldList = Gson().fromJson<List<String>>(untranslatedOldText, ArrayList::class.java)

val translatedOldFile = File(workDirPath, mappingValueFileNameOld)
val translatedOldText = translatedOldFile.readText()
val translatedOldList = Gson().fromJson<List<String>>(translatedOldText, ArrayList::class.java)

val patchedKeyList = mutableListOf<String>()
var unpatchList = mutableListOf<String>()
val patchList = untranslatedList.mapNotNull { key ->
    val oldIndex = untranslatedOldList.indexOf(key)
    if (oldIndex != -1) {
        patchedKeyList.add(key)
        translatedOldList[oldIndex]
    } else {
        // 将未翻译的内容追加到末尾
        unpatchList.add(key)
        null
    }
}
// 写入翻译文件并更新原映射关系
File(workDirPath, mappingKeyFileName).writeText(Gson().toJson(patchedKeyList + unpatchList))
File(workDirPath, mappingValueFileName).writeText(Gson().toJson(patchList + unpatchList))

In [ ]:
// 检查翻译正确性
// todo


In [12]:
// 制作翻译映射表
@file:DependsOn("com.google.code.gson:gson:2.10.1")

import com.google.gson.Gson
import kotlinx.serialization.json.jsonArray
import kotlinx.serialization.json.jsonObject

val untranslatedFile = File(workDirPath, mappingKeyFileName)
val untranslatedText = untranslatedFile.readText()
val untranslatedList = Gson().fromJson<List<String>>(untranslatedText, ArrayList::class.java)

val translatedFile = File(workDirPath, mappingValueFileName)
val translatedText = translatedFile.readText()
val translatedList = Gson().fromJson<List<String>>(translatedText, ArrayList::class.java)

val map = untranslatedList.mapIndexed { index, s ->
    Pair(s, translatedList[index])
}.toMap()

In [13]:
// 开始翻译
// 更新文件夹下所有 .tscn 文件
import java.io.FileFilter

var counter = 0
folder.walk()
    .filter { (it.isFile && it.extension == "tscn") }
    .forEach { tscn ->
        val data = tscn.readText()
        val newData = data.replace(regex) { mr ->
            val newText = map[mr.groupValues[2]]
            if (newText == null) {
                println("找不到{ ${mr.groupValues[2]} }的译文，不做替换")
                mr.value
            } else {
                counter++
                """${mr.groupValues[1]} = "$newText""""
            }
        }
        tscn.writeText(newData)
    }
println("替换完成，共完成 $counter 次替换")

替换完成，共完成 699 次替换


后续操作：替换assets/fonts下的所有 FiraSans 为任意支持中文的字体，注意保持文件名相同

使用以下命令打包
```
.\godotpcktool.exe .\vcb.pck -a a extracted --remove-prefix extracted
```